# Hospital Patient Data Analysis

In [2]:
import pandas as pd
import numpy as np

## Load the Datasets

In [3]:
df_patient = pd.read_csv("Patient_Data (1).csv")
df_billing = pd.read_csv("Billing_Data.csv")



In [4]:
df_patient

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45
5,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00


In [5]:
df_billing

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


### Patient Information 

In [6]:
print(' Patient Dataset Info:')
df_patient.info()

 Patient Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


### Bill Information 


In [7]:
print('\nBilling Dataset Info:')
df_billing.info()


Billing Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


## Selecting relevant columns

In [8]:
df_patient = df_patient[['PatientID', 'Department', 'Doctor', 'BillAmount']]
df_patient.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


### Drop Administrative Columns

In [9]:
df_patient = df_patient.drop(columns=['ReceptionistID', 'CheckInTime'], errors='ignore')
df_patient.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


### Total Bill Amount per Department

In [10]:
dept_total = df_patient.groupby('Department')['BillAmount'].sum().reset_index()
print(' Total Bill Amount per Department:')
print(dept_total)

 Total Bill Amount per Department:
    Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0


### Remove Duplicate Patient Records

In [11]:
df_patient = df_patient.drop_duplicates(subset='PatientID')
df_patient.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


### Fill Missing Bill Amounts with Mean

In [12]:
mean_bill = df_patient['BillAmount'].mean()
df_patient['BillAmount'] = df_patient['BillAmount'].fillna(mean_bill).round(2)
df_patient.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.00
1,102,Neurology,Dr. John,6233.33
2,103,Orthopedics,Dr. Lee,7500.00
3,104,Cardiology,Dr. Smith,6200.00
4,105,Dermatology,Dr. Rose,6233.33


### Merge Billing Dataset with Patient Dataset

In [13]:
df_merged = pd.merge(df_patient, df_billing, on='PatientID')
df_merged.head()

,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.00,2000,3000
1,102,Neurology,Dr. John,6233.33,1500,3500
2,103,Orthopedics,Dr. Lee,7500.00,2500,5000
3,104,Cardiology,Dr. Smith,6200.00,3000,3200
4,105,Dermatology,Dr. Rose,6233.33,1000,4000


### Concatenate New Patients (Row-wise)

In [14]:
new_patients = pd.DataFrame({
    'PatientID': [201, 202],
    'Department': ['Cardiology', 'ENT'],
    'Doctor': ['Dr. Nishi', 'Dr. Mohith'],
    'BillAmount': [4500, 1500]})
new_patients

,PatientID,Department,Doctor,BillAmount
0,201,Cardiology,Dr. Nishi,4500
1,202,ENT,Dr. Mohith,1500


In [15]:
df_final = pd.concat([df_merged, new_patients])
df_final

,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.00,2000.0,3000.0
1,102,Neurology,Dr. John,6233.33,1500.0,3500.0
2,103,Orthopedics,Dr. Lee,7500.00,2500.0,5000.0
3,104,Cardiology,Dr. Smith,6200.00,3000.0,3200.0
4,105,Dermatology,Dr. Rose,6233.33,1000.0,4000.0
0,201,Cardiology,Dr. Nishi,4500.00,NaN,NaN
1,202,ENT,Dr. Mohith,1500.00,NaN,NaN


In [23]:
new_patients = pd.DataFrame({
    'PatientID': [601, 602],
    'Department': ['Cardiology', 'ENT'],
    'Doctor': ['Dr. Koushik', 'Dr. Tarun'],
    'BillAmount': [4500, 3800]
})

df_final = pd.concat([df_merged, new_patients], ignore_index=True)
df_final

,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.00,2000.0,3000.0
1,102,Neurology,Dr. John,6233.33,1500.0,3500.0
2,103,Orthopedics,Dr. Lee,7500.00,2500.0,5000.0
3,104,Cardiology,Dr. Smith,6200.00,3000.0,3200.0
4,105,Dermatology,Dr. Rose,6233.33,1000.0,4000.0
5,601,Cardiology,Dr. Koushik,4500.00,NaN,NaN
6,602,ENT,Dr. Tarun,3800.00,NaN,NaN


### Concatenate New Billing Columns (Column-wise)

In [24]:
new_columns = pd.DataFrame({'InsuranceCovered':np.resize(['Yes', 'No'],len(df_final)),  # repeat to match length
    'FinalAmount': df_final['BillAmount'] * 0.9 # 10% discount applied
})

df_final = pd.concat([df_final, new_columns], axis=1)
df_final.head()

,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.00,2000.0,3000.0,Yes,4500.000
1,102,Neurology,Dr. John,6233.33,1500.0,3500.0,No,5609.997
2,103,Orthopedics,Dr. Lee,7500.00,2500.0,5000.0,Yes,6750.000
3,104,Cardiology,Dr. Smith,6200.00,3000.0,3200.0,No,5580.000
4,105,Dermatology,Dr. Rose,6233.33,1000.0,4000.0,Yes,5609.997


### Final Cleaned Dataset

In [25]:
print(' Final Cleaned Dataset Preview:')
display(df_final.head())

df_final.to_csv('Final_Hospital_Patient_Data.csv', index=False)
print('\n Final dataset saved as: Final_Hospital_Patient_Data.csv')

 Final Cleaned Dataset Preview:


,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.00,2000.0,3000.0,Yes,4500.000
1,102,Neurology,Dr. John,6233.33,1500.0,3500.0,No,5609.997
2,103,Orthopedics,Dr. Lee,7500.00,2500.0,5000.0,Yes,6750.000
3,104,Cardiology,Dr. Smith,6200.00,3000.0,3200.0,No,5580.000
4,105,Dermatology,Dr. Rose,6233.33,1000.0,4000.0,Yes,5609.997



 Final dataset saved as: Final_Hospital_Patient_Data.csv
